# Database engineering

In this section we'll create:

+ table schemas using SQLAlchemy ORM
+ create a database in SQLite
+ load the cleaned Hawaii climate data into pandas dataframes
+ upload the data from the pandas dataframes into the SQLite database

In [1]:
# Dependencies
import pandas as pd
import sqlite3
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, create_session
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.automap import automap_base

In [2]:
# Define and create a database engine 
engine = create_engine('sqlite:///hawaii.sqlite', echo=False)

In [3]:
# Use SQLAlchemy to create a database table schema
Base = declarative_base()

class Station(Base):

    __tablename__ = "station"

    station_id = Column(Integer, primary_key=True)
    station = Column(String, nullable=False)
    name = Column(String, nullable=False)
    latitude = Column(Integer, nullable=False)
    longitude = Column(Integer, nullable=False)
    elevation = Column(Integer, nullable=False)
    
    children = relationship("measurement", back_populates="parent")
    
    def __init__(self, name):

        self.name = name
      
class Measurement(Base):
    
    __tablename__ = "measurement"
    
    measurement_id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Integer)
    tobs = Column(Integer)    
       
    parent = relationship("station", back_populates="parent")
    
    def __init__(self, name):

        self.name = name    

# Generate schema
Base.metadata.create_all(engine)

In [5]:
# Reflect database into a new model
Base = automap_base()
# Reflect tables
Base.prepare(engine)

# Access and reflect metadata 
metadata = MetaData(bind=engine)
metadata.reflect()
# Create database session object
session = create_session(bind = engine)

In [6]:
# Check whether classes and tables exist
for mappedclass in Base.classes:
    print(mappedclass)

for mdtable in Base.metadata.tables:
    print(mdtable)

In [7]:
# Define SQLite connection and cursor
conn = sqlite3.connect("hawaii.sqlite")
cur = conn.cursor()

In [8]:
# Delete any existing table data (for test purposes only)
# https://stackoverflow.com/questions/11233128/how-to-clean-the-database-dropping-all-records-using-sqlalchemy
for tbl in metadata.sorted_tables:
    engine.execute(tbl.delete())
conn.commit()
## Compact SQLite file
conn.execute("VACUUM")

In [9]:
# Load clean data
station_df = pd.read_csv("clean_hawaii_stations.csv")
measurement_df = pd.read_csv("clean_hawaii_measurements.csv")

In [10]:
# Append data to SQLAlchemy tables
station_df.to_sql('station', conn, if_exists='append', index=False)
measurement_df.to_sql('measurement', conn, if_exists='append', index=False)

In [11]:
# Close connection
conn.close()